In [9]:
import nbimporter
import numpy as np
from model_calls import gaussian_psf, bright, mkfields
from astropy.io import fits
from struct import pack
from tqdm import tqdm_notebook, tnrange

In [10]:
field = 35.0/60.0     # field size in deg
resol = 4.0*1024      # resolution of CCD
fwhm = 1.5            # seeing fwhm in arcsec
fields = 'set'        # 'set' to take my fields, number to use that many random fields
resol = int(resol)    # integer pixels
fwhm /= (60.0*60.0)   # seeing fwhm in deg
sigma = fwhm/2.35482  # seeing std in deg
filtername = 'sdss-r'    # prefix for the creation of files
texp = 5.0*60.0       # exposure time in s
dtel = 100.0          # telescope diameter in cm

In [11]:
prefix = filtername
if filtername == 'J-R':
    sfl = 0.278            # star flux in ph/s/cm^2
    sbr = 0.00779          # sky brightness in ph/s/cm^2/arcsec^2
elif filtername == 'sdss-r':
    sfl = 0.225            # star flux in ph/s/cm^2
    sbr = 0.00560          # sky brightness in ph/s/cm^2/arcsec^2
elif filtername == 'sdss-r_skin':
    sfl = 0.210            # star flux in ph/s/cm^2
    sbr = 0.00514          # sky brightness in ph/s/cm^2/arcsec^2

In [12]:
if isinstance(fields, str):
    imstd = "INS\\"+prefix+'_' + \
        str(resol)+'px_'+str(int(field*60))+"'_set_positions"
else:
    imstd = "INS\\"+prefix+'_' + \
        str(resol)+'px_'+str(int(field*60))+"'_"+str(fields)+'stars'

In [13]:
sbright, samp = bright(dtel=dtel, field=field, insky=sbr, instar=sfl,
                       resol=resol, texp=texp)  # sky and star values in ph/px^2 and ph

In [14]:
posx, posy = mkfields(fields, field)  # x and y positions of the stars in deg

In [15]:
# the x,y coordinate system in resol bins but each x,y are measured in deg
x, y = np.meshgrid(np.linspace(0, field, resol), np.linspace(0, field, resol))

In [25]:
shaper = x.shape                                           #
clean = np.empty(shape=shaper)                             # an empty image
#
for i in tqdm_notebook(xrange(len(posx)), desc='Creating stars'):                              #
    # x position of the star in degrees
    xcen = posx[i]
    # y position of the star in degrees
    ycen = posy[i]
    # a gaussian scaled to have total photons as the input star placed in the correct xy position
    curg = gaussian_psf(x, y, sigma, xcen, ycen, samp)
    # add the caussian to the clean image
    clean = np.add(clean, curg)
    #
# add the the sky ph/px^2 to each pixel
pure = np.add(sbright, clean)

In [17]:
division = np.amax(pure)
imapure = np.multiply(np.divide(pure, division), 255).astype(int)
print np.amax(imapure)

255


In [18]:
division = 1.0
bimpure = np.divide(pure, division)
print np.amax(bimpure)

49974.1398288


In [20]:
purehdu = fits.PrimaryHDU(pure)
purehdu.writeto(imstd+".fit")

In [24]:
f = open(imstd+".ima", 'w')
f.write(pack("h", 0))
f.write(pack("h", resol))
f.write(pack("h", 1))
for i in tqdm_notebook(xrange(len(imapure)), desc='Writting IMA'):
    for j in range(0, len(imapure[i])):
        f.write(chr(imapure[i][j]))
f.close()

In [23]:
f = open(imstd+".bim", 'w')
f.write(pack("l", resol))
f.write(pack("l", resol))
for i in tqdm_notebook(xrange(len(bimpure)), desc='Writting BIM'):
    for j in range(0, len(bimpure[i])):
        f.write(pack("d", bimpure[i][j]))
f.close()

In [ ]:
f = open(imstd+".unit", 'w')
totint = np.sum(pure)
maxint = np.amax(pure)
f.write('Total Intensity (ph)\tMaximum Intensity(ph)\n')
f.write(str(totint)+'\t\t\t'+str(maxint))
f.close()